# Lineare und logistische Regression mit Python

## Was ist Regression eigentlich ?

### Kurz und knapp:

Anhand der Regression lässt sich feststellen, ob sich durch eine Variable eine andere erklären lässt und ob es einen Zusammenhang zwischen diesen gibt. Ziel ist es dann, mithilfe der unabhängigen Variablen die abhängige Zielvariable vorherzusagen.
Mögliche Fragestellungen einer Regression wären bspw:

- Steigt die Armut bei Familien, wenn diese mehr Kinder haben ? 

- Sinkt die Anzahl der Temposünder, wenn man mehr Schilder aufstellt ?

- Verbessert sich die Note einer Prüfung, wenn das Bestehen von Scheinaufgaben erforderlich sind für die Teilnahme.

Hierbei wird die lineare Regression verwendet, wenn der Zusammenhang zwischen einer abhängigen Variable und einer oder mehreren unabhängigen Variablen ermittelt werden soll.
Die logistische Regression findet ihren Einsatz, wenn der Zusammenhang zwischen einer abhängigen Variable die Binär ist und unabhängigen Variablen ermittelt werden soll.

## Lineare Regression mit Python

Es gibt mehrere Möglichkeiten, die lineare Regression in Python zu realisieren. Mann kann diese entweder mit numpy, scipy, statsmodel oder sckit-learn implementieren. In diesem Notebook werde ich die lineare Regression mithilfe des mächtigen Pythonmoduls Scikit-learn erklären. 
Der Datensatz den ich verwenden werde, ist das Boston Housing Dataset (Quelle: UCI Machine Learning Repository). Der Datensatz enthält Daten zum Immobilienwert in den Vororten von Boston.


Zuerst müssen die benötigten Pakete importiert werden. 

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn

Der Datensatz den wir verwenden möchten, ist bereits im Modul sklearn vorhanden, und muss deswegen einfach nur geladen werden. Den Datensatz wird anschließend im Objekt boston im Datentyp Dictionary gespeichert.

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

In der Beschreibung des Datensatzes sehen wir, dass wir 13 unabhängige Variablen haben und eine abhänginge Variable (MEDV). Die abhängige Variable wird auch Target genannt. Wir werden jetzt versuchen, mithilfe der 13 Variablen den Preis vorherzusagen.

In [ ]:
x = boston.DESCR
print(x)

Zuerst laden wir die Daten in ein Dataframe.

In [ ]:
bos = pd.DataFrame(boston.data)
bos.head()

Weil das Dataframe die Variablennamen noch nicht enthält, ersetzen wir die Nummerierung mit den Variablennamen.


In [ ]:
bos.columns = boston.feature_names
bos.head

Die Variable die den Preis enthält ist in unserem Dataframe nicht zu finden. Wir holen die Preise aus boston.target und hängen sie dann an das Dataframe an.

In [ ]:
bos['PRICE'] = boston.target

#### Lineare Regression mit Scikit-Learn

Um die Preise vorhersagen zu können, müssen wir nun ein lineares Regressionsmodel anpassen. Dazu verwenden wir die LinearRegression.fit Methode von Scikit. Dieser Methode müssen zwei Variablen übergeben werden, damit sie das Modell anpassen kann: 

- Y : Der Target (in diesem Fall der Preis)
- X : Die unabhängige/n Variable/n (in diesem Fall wären dass die 13 übrigen Variablen)

In [ ]:
from sklearn.linear_model import LinearRegression
X = bos.drop('PRICE', axis = 1) # weil wir alle Variablen außer den Preis in X haben wollen

# Hier erzeugen wir ein Objekt von LinearRegression

lr = LinearRegression()
lr.fit(X,bos.PRICE)

Jetzt können wir uns die geschätzten Koeffizienten und die Konstante unserer Linearen Gleichung anzeigen lassen.

In [ ]:
konstante = lr.intercept_
koeffizienten = lr.coef_
print(konstante)
print(koeffizienten)

Zur besseren Visualisierung unserer Konstanten erstellen wir einen Dataframe, wo wir an die 13 Variablen den Koeffizienten anhängen. 

In [ ]:
pd.DataFrame = list(zip(X.columns, lr.coef_))
pd.DataFrame

Das gelernte Regressionsmodell ist also die lineare Gleichung:

Preis = -0,11 CRIM 0,05 ZN 0,02 INDUS... -0,53 LSTAT +36,49

Zudem sieht man nun, dass RM die höchste Korrelation zum Target besitzt. Das lässt sich mit einem Scatterplot mit RM und Preis noch besser verdeutlichen.

In [ ]:
plt.scatter(bos.RM, bos.PRICE)
plt.xlabel("RM")
plt.ylabel("Preis")
plt.show

Nun werden wir den Preis vorhersagen. Um zu schauen, ob unser lineares Regressionsmodell gut genug ist, werden wir in einem Scatterplot die vorhergesagten Preise und die tatsächlichen Preise miteinander vergleichen. Mit der LinearRegression-Methode .predict lässt sich eine Vorhersage berechnen.

In [ ]:
plt.scatter(bos.PRICE, lr.predict(X))
plt.xlabel("Preis: $Y_i$")
plt.ylabel("Vorhergesagter Preis: $\hat{Y}_i$")
plt.show

Hier lässt sich jetzt unschwer erkennen, dass wir falsch geschätzte Werte haben im oberen Preissegment. Das lässt sich zurückführen auf den MSE (mean squared error). Dieser ist in der Regel unvermeidbar und gibt an, wie gut unsere geschätzten Werte die tatsächlichen Werte treffen. Je höher der MSE ist, umso ungenauer ist die Vorhersage. Den MSE werden wir nun berechnen.

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(bos.PRICE, lr.predict(X))
print(mse)




Jetzt werden wir ein lineares Regressionsmodell für nur eine Variable aufstellen und den MSE mit dem des Modells über die 13 Variablen vergleichen.

In [ ]:
lf= LinearRegression()
lf.fit(X[['PTRATIO']],bos.PRICE)
mean_squared_error(bos.PRICE, lf.predict(X[['PTRATIO']]))


Der MSE hat sich mehr als verdoppelt, was bedeutet, dass eine Variable alleine keine gute Vorhersage machen kann. In einem Scatterplot, wo wir den Preis mithilfe des neuen Modells vorhersagen, sieht man noch deutlicher, wie ungenau diese Methode ist.

In [ ]:
plt.scatter(bos.PRICE, lf.predict(X[['PTRATIO']]))
plt.xlabel("Preis: $Y_i$")
plt.ylabel("Vorhergesagter Preis: $\hat{Y}_i$")
plt.show

### Training und Test

Das Modell das wir gerade aufgebaut haben ist in der Form jedoch unbrauchbar, weil wir für die Erstellung des Modells die selben Daten verwendet haben, auf die wir das Modell dann angewendet haben für die Vorhersage des Preises.
Normalerweise soll ein Modell auf neue Daten angewendet werden und dort möglichst genaue Prognosen erzielen.

Aus diesem Grund wird der Datensatz ein einen Trainigsdatensatz und einen Testdatensatz gesplitet. Mit dem Trainingsdatensatz stellen wir unser Modell auf und am Testdatensatz sehen wir dann, wie gut unsere Regression ist.

Theoretisch könnte man jetzt hergehen und den Boston-Datensatz einfach in der Mitte in Trainings- und Testdatensatz teilen. Das Problem, dass daraus resultieren könnte ist jedoch, dass man dann wahrscheinlich das Modell mit den billigen Immobilien aufstellt und dann an den teuren testet. Natürlich liegt auf der Hand, dass so ein Modell nicht genaus sein wird und deswegen wird der Datensatz zufällig geteilt.

Scikit enthält die Funktion train_test_split, die den Trainings- und Testdatensatz zufällig erzeugen kann.

In [ ]:
from sklearn.cross_validation import train_test_split
X_training, X_test, Y_training, Y_test = sklearn.cross_validation.train_test_split(
X, bos.PRICE, test_size=0.4, random_state = 4)
X_test


Nun stellen wir dass Modell mit dem Trainingsdatensatz auf und machen auf dem Testdatensatz eine Vorhersage.

In [ ]:
lt = LinearRegression()
lt.fit(X_training, Y_training)
predicted = lt.predict(X_test)

Jetzt berechnen wir den MSE und vergleichen diesen mit dem des Modells über die gesamten Daten (21.8977792177)

In [49]:
from sklearn import metrics 

mse = mean_squared_error(Y_test, predicted)
print(mse)



NameError: name 'Y_test' is not defined

Der MSE ist nun höher als beim Modell über die gesamten Daten. Im Gegensatz zum MSE des ersten Ansatzes ist dieser jedoch gültig, da das Modell auf einem "neuen" Datensatz angewendet wurde.

## Logistische Regression mit Python

Jetzt wo wir wissen, wie und wann man die lineare Regression anwendet, können wir uns der logistischen Regression widmen. 
Wie bereits erwähnt wird auch diese dafür verwendet, Korrelationen zwischen abhängigen und unabhängigen Variablen zu ermitteln und Vorhersagen zu treffen.
Der Unterschied ist, dass die logistische Regression nur dann zum Einsatz kommt, wenn das Target (abhängige Variable) binär ist.

Mithilfe von scikit-learn werden wir nochmal einen Datensatz analysieren und eine logistische Regression durchführen. Der Datensatz den wir benutzen werden ist das 'affairs dataset', welches in Statsmodels vorhanden ist und nur importiert werden muss. Es ist das Ergebnis einer Umfrage von verheirateten Frauen, welche über außereheliche Beziehungen befragt wurden.





Der erste Schritt ist wieder das Importieren der benötigten Module. 

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Jetzt laden wir das Dataset herunter. Es enthält 6366 Zeilen mit 9 Variablen: 

- rate_marriage: Bewertung der Ehe durch Ehefrau (1 = sehr schlecht, 5 = sehr gut)
- age: Alter der Frau
- yrs_married
- children
- religious: Selbständige Bewertung der Religiösität der Frau (1 = gar nicht, 4 sehr religös)
- educ: Bildungsstand (9 = kein High-school Abschluss , 12 = High-school Abschluss, 14 = College, 16 = College Abschluss, 17 = Akademikerin, 20 = PhD etc.)
- occupation: Beruf der Frau (1 = Schülerin/Studentin, 2 = Hilfsarbeiterin, 3 = Facharbeiterin, 4 = Lehrerin,Schriftstellerin,Krankenschwester..., 5 = Management, 6 = Vollakademikerin)
- occupation_husb: Beruf des Ehemanns (selbe Codierung)
- affairs: Dauer der außerehelichen Beziehungen

In [3]:
dta = sm.datasets.fair.load_pandas().data

In [4]:
dta.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0.111111
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3.230769
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1.400000
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0.727273
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4.666666


Unser Target wird die binäre Variable 'affaere' sein, die wir noch erstellen müssen. 0 steht für keine Affäre und 1 für Affäre.

In [5]:
dta['affaere'] = (dta.affairs > 0).astype(int)

In [27]:
dta

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs,affaere
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0.111111,1
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3.230769,1
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1.400000,1
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0.727273,1
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4.666666,1
5,4.0,27.0,9.0,0.0,2.0,14.0,3.0,4.0,4.666666,1
6,5.0,37.0,23.0,5.5,2.0,12.0,5.0,4.0,0.852174,1
7,5.0,37.0,23.0,5.5,2.0,12.0,2.0,3.0,1.826086,1
8,3.0,22.0,2.5,0.0,2.0,12.0,3.0,3.0,4.799999,1
9,3.0,27.0,6.0,0.0,1.0,16.0,3.0,5.0,1.333333,1


Bevor wir das Regressionsmodell aufstellen, bereiten wir unsere Daten etwas auf. Die Variablen 'occupation' und 'occupation_husb' werden wir als kategorische Variablen betrachten. Mit der Funktion dmatrices lässt sich das implementieren.

Zudem definieren wir mit der Funktion dmatrices 'affaere' als Y und die restlichen Variablen mit Ausnahme von 'affairs' und 'affaere' als X.

Zur besseren Visualisierung vergeben wir in diesem Schritt auch noch neue Namen für die neu erstellten kategorischen Variablen.


In [35]:
y, X = dmatrices('affaere ~ rate_marriage + age + yrs_married + children + \
                  religious + educ + C(occupation) + C(occupation_husb)',
                  dta, return_type="dataframe")
y = np.ravel(y) # y machen wir zu einem 1-dimensionalen Array, damit es als Target von Scikit-learn richtig erkannt wird

X = X.rename(columns = {'C(occupation)[T.2.0]':'occ_2',
                        'C(occupation)[T.3.0]':'occ_3',
                        'C(occupation)[T.4.0]':'occ_4',
                        'C(occupation)[T.5.0]':'occ_5',
                        'C(occupation)[T.6.0]':'occ_6',
                        'C(occupation_husb)[T.2.0]':'occ_husb_2',
                        'C(occupation_husb)[T.3.0]':'occ_husb_3',
                        'C(occupation_husb)[T.4.0]':'occ_husb_4',
                        'C(occupation_husb)[T.5.0]':'occ_husb_5',
                        'C(occupation_husb)[T.6.0]':'occ_husb_6'})
print(X.columns)

Index(['Intercept', 'occ_2', 'occ_3', 'occ_4', 'occ_5', 'occ_6', 'occ_husb_2',
       'occ_husb_3', 'occ_husb_4', 'occ_husb_5', 'occ_husb_6', 'rate_marriage',
       'age', 'yrs_married', 'children', 'religious', 'educ'],
      dtype='object')


Nun können wir das logistische Regressionsmodell aufstellen. Wie vorhin werden wir dieses zuerst über den ganzen Datensatz aufstellen.


In [36]:
logmodell = LogisticRegression()
logmodell = logmodell.fit(X, y)
pred_log = logmodell.predict(X)

Jetzt schauen wir uns an, wie genau unsere Regression ist. Diesmal werden wir nicht wie vorhin den MSE berechnen, sondern die Methode score benutzen, die wenn sie X und Y als Parameter übergeben bekommt, die mitllere Genauigkeit der geschätzten Werte zurückgibt.

In [38]:
from sklearn.metrics import mean_squared_error 

mse = mean_squared_error(y, pred_log)
mse




0.27411247251021048

Wie wir aber bereits bei der linearen Regression gelernt haben, ist dieser Wert ungültig, weil das Modell auf den selben Daten aufgestellt wurde, auf die es angewendet wurde. 

Um einen gültigen Wert zu erzielen, müssen wir wieder die Methode mit dem Trainings- und Testdatensatz anweden. Wir splitten den Datensatz wieder mit der Funktion train_test_split.

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logmodell2 = LogisticRegression()
logmodell2.fit(X_train, y_train)
pred_log = logmodell2.predict(X_test)
mse = mean_squared_error(y_test, pred_log)
print(mse)


0.270157068063


In diesem Fall ist das Modell genauso präzise wie das Modell über die ganzen Daten. 


Außer den MSE gibt es noch eine weitere Möglichkeit die Genauigkeit eines Modells festzustellen. Scikit-learn bietet die Funktion .score an, die die mittlere Genauigkeit der vorhergesagten Werte angibt.

In [31]:
logmodell2.score(X,y)

0.72573044297832234

Die Genauigkeit der Vorhersagen liegt bei 72 %

Um uns vor Augen zu bringen, was man mit diesem Modell anstellen kann, werden wir nun die Wahrscheinlichkeit vorhersagen, dass eine fiktive Studienteilnehmerin eine außereheliche Beziehung hat. Die fiktive Frau ist eine 60 Jahre alte Lehrerin, die seit 30 Jahren verheiratet ist, 3 Kinder hat, religös ist, ihre Ehe als sehr gut bezeichnet und meinem einem Landwirt verheiratet ist.

In [25]:
logmodell2.predict_proba(np.array([1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 60, 30, 3, 4,
                              16]))

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.24710507,  0.75289493]])

Die Wahrscheinlichkeit, dass sich besagte fiktive Frau schonmal einen Fehltritt in der Ehe geleistet hat, liegt mit 72%-Wahrscheinlichkeit bei 75%

### Quellen:

- http://www.dataschool.io/logistic-regression-in-python-using-scikit-learn/cliqz.com/tracking/
- https://de.wikipedia.org/wiki/Regressionsanalyse
- https://en.wikipedia.org/wiki/Linear_regression
- https://en.wikipedia.org/wiki/Logistic_regression
